In [2]:
!pip install rouge

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from rouge import Rouge

# Load dataset
data = pd.read_csv("Reviews.csv", nrows=5000)
# Drop Duplicates and NA values
data.drop_duplicates(subset=['Text'], inplace=True)  # dropping duplicates
data.dropna(axis=0, inplace=True)  # dropping na
data = data.reset_index(drop=True)


# Preprocessing
def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words]
    return " ".join(tokens)

data['Text'] = data['Text'].apply(preprocess_text)

# Tokenization
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(data['Text'])
vocab_size = len(tokenizer.word_index) + 1

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(data['Text'], data['Summary'], test_size=0.2, random_state=42)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

# Pad sequences
max_length = 100  # Assuming maximum sequence length
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')

# REINFORCE model
class REINFORCE(tf.keras.Model):
    def __init__(self, num_actions, vocab_size):
        super(REINFORCE, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, 128, input_length=max_length)
        self.lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))
        self.global_pooling = tf.keras.layers.GlobalMaxPooling1D()
        self.dense = tf.keras.layers.Dense(64, activation='relu')
        self.output_layer = tf.keras.layers.Dense(num_actions, activation='softmax')

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.lstm(x)
        x = self.global_pooling(x)
        x = self.dense(x)
        return self.output_layer(x)

# Custom loss function
def REINFORCE_loss(y_true, y_pred, rewards):
    return -tf.reduce_mean(tf.math.log(y_pred) * rewards)

# Optimizer
optimizer = tf.keras.optimizers.Adam()

# Training function
@tf.function
def train_step(model, inputs, rewards):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = REINFORCE_loss(None, predictions, rewards)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Example usage
num_actions = vocab_size  # Number of actions (size of vocabulary for text summarization)
reinforce_model = REINFORCE(num_actions, vocab_size)

# Training loop
NUM_EPOCHS = 3
gamma = 0.99  # Discount factor
rouge = Rouge()

for epoch in range(NUM_EPOCHS):
    epoch_loss_avg = tf.keras.metrics.Mean()
    for text, summary in zip(X_train, y_train):  # Iterate over original text, not padded sequences
        print("INPUT TEXT:")
        print(text)
        print("-----------")
        state = tokenizer.texts_to_sequences([text])[0]  # Convert text to sequence
        state_text = ' '.join([tokenizer.index_word.get(idx, 'UNK') for idx in state])  # Convert sequence back to text
        print("Input sequence (text):", state_text)  # Print input text
        state_padded = tf.keras.preprocessing.sequence.pad_sequences([state], maxlen=max_length, padding='post', truncating='post')  # Pad sequence
        if np.isnan(state_padded).any():
            print("NaN values found in input sequence:", state_padded)
            continue
        with tf.GradientTape() as tape:
            action_probs = reinforce_model(state_padded)
            action = tf.random.categorical(tf.math.log(action_probs), num_samples=1)[0, 0].numpy()
            generated_summary = tokenizer.index_word.get(action, 'UNK')
            print("Generated summary:", generated_summary)  # Debug print
            reward = rouge.get_scores(generated_summary, summary)[0]['rouge-1']['f']
            print("Raw reward (ROUGE score):", reward)  # Debug print
            reward = max(min(reward, 1.0), -1.0)  # Reward clipping
            reward = (reward - np.mean(reward)) / (np.std(reward) + 1e-8)  # Normalize reward
            print("Normalized reward:", reward)
            loss = train_step(reinforce_model, state_padded, tf.constant(reward, dtype=tf.float32))
            print("Loss:", loss)
            epoch_loss_avg.update_state(loss)
    print("Epoch {}: Loss: {:.4f}".format(epoch + 1, epoch_loss_avg.result()))


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Streaming output truncated to the last 5000 lines.
perfect baby toddler go started using baby version daughter infant easy throw pouch baby spoon diaper bag daughter older able feed pouch give toddler pouch fruit packaging great fruit taste good well yes tasted everything feed daughter daughter love whole face light hand pouch
-----------
Input sequence (text): perfect baby toddler go started using baby version daughter infant easy throw pouch baby spoon diaper bag daughter older able feed pouch give toddler pouch fruit packaging great fruit taste good well yes tasted everything feed daughter daughter love whole face light hand pouch
Generated summary: fatigue
Raw reward (ROUGE score): 0.0
Normalized reward: 0.0
Loss: tf.Tensor(0.0, shape=(), dtype=float32)
INPUT TEXT:
mom kid older two hard time finding snack much sodium came along happyfamily introduced happymunchies thankful find snack low sodium vegetable top organic know free pesticide residue gmos
-----------
Input sequence (text

In [13]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from rouge import Rouge

# Load dataset
data = pd.read_csv("Reviews.csv", nrows=5000)
# Drop Duplicates and NA values
data.drop_duplicates(subset=['Text'], inplace=True)  # dropping duplicates
data.dropna(axis=0, inplace=True)  # dropping na
data = data.reset_index(drop=True)


# Preprocessing
def preprocess_text(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words]
    return " ".join(tokens)

data['Text'] = data['Text'].apply(preprocess_text)

# Tokenization
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(data['Text'])
vocab_size = len(tokenizer.word_index) + 1

# Train-test split
X_train, X_val, y_train, y_val = train_test_split(data['Text'], data['Summary'], test_size=0.2, random_state=42)

# Convert text to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_val_seq = tokenizer.texts_to_sequences(X_val)

# Pad sequences
max_length = 100  # Assuming maximum sequence length
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=max_length, padding='post', truncating='post')

# REINFORCE model
class REINFORCE(tf.keras.Model):
    def __init__(self, num_actions, vocab_size):
        super(REINFORCE, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, 128, input_length=max_length)
        self.lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True))
        self.global_pooling = tf.keras.layers.GlobalMaxPooling1D()
        self.dense = tf.keras.layers.Dense(64, activation='relu')
        self.output_layer = tf.keras.layers.Dense(num_actions, activation='softmax')

    def call(self, inputs):
        x = self.embedding(inputs)
        x = self.lstm(x)
        x = self.global_pooling(x)
        x = self.dense(x)
        return self.output_layer(x)

# Custom loss function
def REINFORCE_loss(y_true, y_pred, rewards):
    return -tf.reduce_mean(tf.math.log(y_pred) * rewards)

# Optimizer
optimizer = tf.keras.optimizers.Adam()

# Training function
@tf.function
def train_step(model, inputs, rewards):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = REINFORCE_loss(None, predictions, rewards)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss

# Example usage
num_actions = vocab_size  # Number of actions (size of vocabulary for text summarization)
reinforce_model = REINFORCE(num_actions, vocab_size)

losses=[]
epoch_losses = []
# Training loop
NUM_EPOCHS = 3
gamma = 0.99  # Discount factor
rouge = Rouge()

for epoch in range(NUM_EPOCHS):
    epoch_loss_avg = tf.keras.metrics.Mean()
    for text, summary in zip(X_train, y_train):  # Iterate over original text, not padded sequences
        print("INPUT TEXT:")
        print(text)
        print("-----------")
        state = tokenizer.texts_to_sequences([text])[0]  # Convert text to sequence
        state_text = ' '.join([tokenizer.index_word.get(idx, 'UNK') for idx in state])  # Convert sequence back to text
        #print("Input sequence (text):", state_text)  # Print input text
        state_padded = tf.keras.preprocessing.sequence.pad_sequences([state], maxlen=max_length, padding='post', truncating='post')  # Pad sequence
        if np.isnan(state_padded).any():
            print("NaN values found in input sequence:", state_padded)
            continue
        with tf.GradientTape() as tape:
            action_probs = reinforce_model(state_padded)
            action = tf.random.categorical(tf.math.log(action_probs), num_samples=1)[0, 0].numpy()
            generated_summary = tokenizer.index_word.get(action, 'UNK')
            print("Generated summary:", generated_summary)  # Debug print
            print("Original summary:",summary)
            reward = rouge.get_scores(generated_summary, summary)[0]['rouge-1']['f']
            reward = max(min(reward, 1.0), -1.0)  # Reward clipping
            reward = (reward - np.mean(reward)) / (np.std(reward) + 1e-8)  # Normalize reward
            loss = train_step(reinforce_model, state_padded, tf.constant(reward, dtype=tf.float32))
            epoch_losses.append(loss)
    epoch_loss_avg = np.mean(epoch_losses)
    losses.append(epoch_loss_avg)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Streaming output truncated to the last 5000 lines.
INPUT TEXT:
like better regular altoids even costly better like
-----------
Generated summary: hunger
Original summary: Nice little mints, but pricey.
INPUT TEXT:
kid love brownie could longer regular brand grocery store good conscious still wanted convenience mix though fit bill first time made thought tasted funny used processed crap eat gotten used flax taste think yummy br way though instruction call lot oil egg butter use earth balance egg replacer vegan flax seed also good amount fat although healthy kind made olive oil good also added extra organic cocoa powder make extra chocolatey really need love chewy texture flax seed ingredient could probably make brownie homemade would pain grind flax seed etc easy yummy even though necessarily healthy calorie fat standpoint feel good giving kid treat loaded processed crap
-----------
Generated summary: substitute
Original summary: Healtier, but not guilt-free...
INPUT TEXT:
hoping good i